In [1]:
from __future__ import print_function, unicode_literals
import pickle

import pandas as pd
import datetime
from tqdm import tqdm
import csv
from pandas import Series
from datetime import datetime
import random
import numpy as np
import time 
import os
import statsmodels.api as sm
import pdb
from datetime import date
import sys
from data_provider.datafeed.quote_feed import QuoteFeed
from data_provider.nestlib.trading_cal import TradeCal
from data_provider.datafeed.universe import Universe
from data_provider.nestlib.market_info import Frequency
from data_provider.datafeed.financial_feed import FinancialFeed
from data_provider.datafeed.finance_utils import ttmContinues, ttmDiscrete
from smartbeta.smartfactor import SmartFactor
from index_li import mao_40_li
from index_li import institutions_baotuan_50_li
import matplotlib.pyplot as plt
uni = Universe()
tc = TradeCal()

tc_handle = TradeCal()
uni_handle = Universe()

now_day = date.today().strftime('%Y%m%d')
if tc_handle.is_trading_day(now_day) == True:
    print('获取'+now_day+'数据。。。')
else:
    print('今天不是交易日 获取数据失败！')
    sys.exit()

获取20210512数据。。。


In [2]:
def get_new_ipo_stocks(date):
    ipo_df = uni.get_all_ipo_info()
    limit = tc.shift_date(date, 30, direction='backward')
    new_ipo_stocks_df = ipo_df[ipo_df['ipo_date']>int(limit)]
    new_ipo_stocks_li = new_ipo_stocks_df['ticker'].tolist()
    return new_ipo_stocks_li

In [3]:
def get_number_of_all_stocks(date):
#     print('正在获取每日股票总数(除去新股)。。。。。。')
    all_stocks_li = uni.get_a_share_by_date(date)
    all_stocks_li = list(set(all_stocks_li)-set(get_new_ipo_stocks(date)))
    return all_stocks_li

In [4]:
def get_number_of_up_stocks_all_stcoks(begin_date,end_date):
    print('正在获取每日上涨的股票数和每日股票总数(除去新股)。。。。。。')
    df = SmartFactor('dailyreturn').load(begin_date,end_date)
    gb_df_data = df.groupby('tdate')
    date_number_li = []
    for any_date,date_df in tqdm(gb_df_data):
        date_df = date_df.copy()
        date_df = date_df[date_df['factor_value']>0]
        date_df = date_df.drop(date_df[date_df['security_code'].isin(get_new_ipo_stocks(any_date))].index)
        number = len(date_df)
        number_all = len(get_number_of_all_stocks(any_date))
        date_number_li.append([any_date,number,number_all])
    result = pd.DataFrame(date_number_li,columns=['date','number_of_up_stocks','number_of_all_stocks'])
    return result

In [5]:
def get_index_dailyreturn(ticker,begin_date,end_date):    
    tickers = ticker
    shift_back_begin_date = tc.shift_date(begin_date, 1, direction='backward')
    week_quote = QuoteFeed(
        universe_ticker=tickers,
        begin_day=shift_back_begin_date,
        end_day=end_date,
        tracking_freq=86400,
        is_index=True,
    )
    week_quote.load_feed()
    df = week_quote.get_index_quote()
    df.loc[:,'yesterday_close'] = df['close'].shift(1)
    df = df.dropna()
    df.loc[:,'dailyreturn'] = (df['close']-df['yesterday_close'])/df['yesterday_close']
    return df[['datetime_str','dailyreturn']]

In [6]:
def get_median_dailyrertun_of_all_market(begin_date,end_date):
    all_stocks_df = SmartFactor('dailyreturn').load(begin_date,end_date)
    gb_df_data = all_stocks_df.groupby('tdate')
    zz500_df = get_index_dailyreturn('000905.SH',begin_date,end_date)
    hs300_df = get_index_dailyreturn('000300.SH',begin_date,end_date)
    date_return_li = []
    for any_date,date_df in tqdm(gb_df_data):
        date_df = date_df.copy()
        median_return = date_df['factor_value'].median()
        hs_300_return = hs300_df.loc[hs300_df[hs300_df['datetime_str']==any_date].index,'dailyreturn'].values[0]
        zz_500_rertun = zz500_df.loc[zz500_df[zz500_df['datetime_str']==any_date].index,'dailyreturn'].values[0]
        date_return_li.append([any_date,median_return,hs_300_return,zz_500_rertun])
    result = pd.DataFrame(date_return_li,columns=['date','all_market_median_return','hs_300_return','zz_500_return'])
    return result    

In [7]:
def get_special_index_return(ticker_li,begin_date,end_date):
    all_stocks_df = SmartFactor('dailyreturn').load(begin_date,end_date)
    special_index_df = all_stocks_df.loc[all_stocks_df[all_stocks_df['security_code'].isin(ticker_li)].index,:]
    data = special_index_df.groupby('tdate')
    return_li = []
    for any_date,date_df in data:
        mean_return = date_df['factor_value'].mean()
        return_li.append([any_date,mean_return])
    return_df = pd.DataFrame(return_li,columns=['date','special_index_mean_dailyreturn'])
    return return_df

In [8]:
def get_open_auction_amount(begin_date,end_date):
    trading_date_li = tc.get_trading_day_list(begin_date,end_date)
    dir_path = '/home/jovyan/work/share/call_auction_data/'
    data_li = []
    ticker_number_li = []
    for td in tqdm(trading_date_li):
        ticker_number = 0
        abs_path = dir_path + td + '.pkl'
        if not os.path.exists(abs_path):
            continue

        if os.path.getsize(abs_path)/(1024*1024.0) < 8:  # 小于8M的数据认为是异常数据
            continue


        with open(dir_path+'{0}.pkl'.format(td), 'rb') as fp:
            tick_dict = pickle.load(fp)

        geye_ret = {}
        for tk, tick_ls in tick_dict.items():
            open_tick = None

            for idx in range(len(tick_ls)):

                if tick_ls[idx]['nTime']>93000000:
                    break

                if tick_ls[idx]['nTime']>=92000000 and tick_ls[idx]['nTime']<=92457000:
                    open_tick = tick_ls[idx]
                    ticker_number = ticker_number + 1

        ticker_number_li.append([td,ticker_number])
    df_ticker = pd.DataFrame(ticker_number_li,columns=['date','open_auction_ticker_number'])
    return df_ticker

In [9]:
def get_all_market_amount(begin_date,end_date):
    all_tickers = uni.get_a_share_in_period(begin_date, end_date)
    quote = QuoteFeed(
    universe_ticker=all_tickers,
    begin_day=begin_date,
    end_day=end_date,
    tracking_freq=86400,
    use_cache=True,
    )
    quote.load_feed()
    all_df = quote.get_stock_quote()
    data = all_df.groupby('datetime')
    amount_li = []
    for any_date,date_df in data:
        total_amount = date_df['amount'].sum()
        mao_40_amount =date_df.loc[date_df[date_df['ticker'].isin(mao_40_li)].index,'amount'].sum()
        institutions_baotuan_50_amount =date_df.loc[date_df[date_df['ticker'].isin(institutions_baotuan_50_li)].index,'amount'].sum()
        amount_li.append([any_date,mao_40_amount,institutions_baotuan_50_amount,total_amount])
    result = pd.DataFrame(amount_li,columns=['date','mao_40_amount','institutions_baotuan_50_amount','all_stocks_total_daily_amount'])
    result.loc[:,'mao_40_ratio'] = result['mao_40_amount']/result['all_stocks_total_daily_amount']
    result.loc[:,'institutions_baotuan_50_ratio'] = result['institutions_baotuan_50_amount']/result['all_stocks_total_daily_amount']
    return result

In [10]:
def get_limit_up_down_number(begin_date,end_date):
    all_df = uni.get_limit_up_down(begin_date,end_date)
    gb_data = all_df.groupby('time')
    data_li = []
    for day,day_df in gb_data:
        day_df = day_df.copy()
        zt_number = len(day_df[(day_df['iszt']==1)|(day_df['iszt2']==1)])
        dt_number = len(day_df[(day_df['isdt']==1)|(day_df['isdt2']==1)])
        data_li.append([day,zt_number,dt_number])
    zt_dt_number_df = pd.DataFrame(data_li,columns=['date','zhangting_number','dieting_number'])
    return zt_dt_number_df

In [11]:
# def get_fin_sec_info(begin_date,end_date):
#     tradeday_li = tc.get_trading_day_list(begin_date,end_date)
#     df = pd.read_csv('fin_sec_info.csv')
#     result = df.loc[df[df['date'].isin(tradeday_li)].index,:]
#     result = result.sort_values(by='date',ascending=False)
#     return result[['date','finval_sum','secval_sum','finval_secval_sum','fin_sec_ratio']] #日期,融资余额,融券余额,融资融券余额,融资占融资融券余额比例

In [12]:
from datetime import datetime
today = datetime.now().strftime('%Y%m%d')
begin_day = today
end_day = today

In [13]:
df1 = get_number_of_up_stocks_all_stcoks(begin_day,end_day)

df2 = get_index_dailyreturn('000905.SH',begin_day,end_day)
df2 = df2.rename(columns={'dailyreturn':'zz500_dailyreturn'})

df3 = get_index_dailyreturn('000300.SH',begin_day,end_day)
df3 = df3.rename(columns={'dailyreturn':'hs300_dailyreturn'})

df4 = get_median_dailyrertun_of_all_market(begin_day,end_day)

df5 = get_special_index_return(institutions_baotuan_50_li,begin_day,end_day)
df5 = df5.rename(columns={'special_index_mean_dailyreturn':'institutions_baotuan_50_mean_dailyreturn'})

df6 = get_special_index_return(mao_40_li,begin_day,end_day)
df6 = df6.rename(columns={'special_index_mean_dailyreturn':'mao_40_mean_dailyreturn'})

df7 = get_all_market_amount(begin_day,end_day)

# df8 = get_fin_sec_info('20160325','20210412')

df9 = get_limit_up_down_number(begin_day,end_day)

df10 = get_open_auction_amount(begin_day,end_day)

df2 = df2.rename(columns={'datetime_str':'date'})
df3 = df3.rename(columns={'datetime_str':'date'})
df7['date'] = df7['date'].apply(lambda x:str(x).replace('-','').split(' ')[0])
# df8['date'] = df8['date'].apply(lambda x: str(x))
df9['date'] = df9['date'].apply(lambda x:str(x).replace('-','').split(' ')[0])

正在获取每日上涨的股票数和每日股票总数(除去新股)。。。。。。


  0%|          | 0/1 [00:00<?, ?it/s]

loading dailyreturn time cost 1.41s


100%|██████████| 1/1 [00:00<00:00,  2.75it/s]
/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


loading dailyreturn time cost 1.42s


100%|██████████| 1/1 [00:00<00:00, 87.52it/s]


loading dailyreturn time cost 1.20s
loading dailyreturn time cost 1.17s
正在使用hdf5行情！


100%|██████████| 1/1 [00:00<00:00,  1.43it/s]


In [14]:
merged_df = pd.merge(df1,df2,on=['date'])
for df in [df3,df4,df5,df6,df7,df9,df10]:
    merged_all_df = pd.merge(merged_df,df,on=['date'],how='outer')
    merged_df = merged_all_df
merged_all_df = merged_all_df.sort_values(by='date',ascending=False)

In [15]:
ori_df = pd.read_pickle('index_monitor.pkl')
update_df = ori_df.append(merged_all_df)
update_df = update_df.sort_values(by='date',ascending=False)
update_df = update_df.drop_duplicates(keep='first')
update_df.to_pickle('index_monitor.pkl')